# Advent of Code 2018

In [1]:
import os
import re

from collections import Counter
from itertools import accumulate, combinations, cycle

import numpy as np
import requests

In [2]:
def input_for_day(day):
    file_name = f"day{day:0>2}_input"
    if os.path.exists(file_name):
        return os.path.abspath(file_name)
    with open("session", "r") as session_file:
        response = requests.get(f"https://adventofcode.com/2018/day/{day}/input",
                                cookies={"session": session_file.read()})
    with open(file_name, "w") as input_file:
        print(response.text.strip(), file=input_file)
    return os.path.abspath(file_name)

## Day 1
### Part One
Starting with a frequency of zero, what is the resulting frequency after all of the changes in frequency have been applied?

In [44]:
%time
with open(input_for_day(1), "r") as input_file:
    print(f'Answer: {sum(map(int, input_file))}')

Answer: 538
Wall time: 998 µs


### Part Two
What is the first frequency your device reaches twice?

In [48]:
%%time
freq_set = {0}
with open(input_for_day(1), "r") as input_file:
    for freq in accumulate(cycle(int(line) for line in input_file)):
        if freq in freq_set or freq_set.add(freq):
            print(f"Answer: {freq}")
            break

Answer: 77271
Wall time: 27 ms


## Day 2
### Part One
What is the checksum for your list of box IDs?

In [56]:
%%time
with open(input_for_day(2), "r") as input_file:
    counters = [Counter(n.strip()).values() for n in input_file]
checksum = sum(2 in c for c in counters) * sum(3 in c for c in counters)
print(f"Answer: {checksum}")

Answer: 5976
Wall time: 2 ms


### Part Two
What letters are common between the two correct box IDs?

In [49]:
%%time
with open(input_for_day(2), "r") as input_file:
    counters = {x: Counter(x).values() for x in (n.strip() for n in input_file)}
for x, y in combinations((n for n, c in counters.items() if any(s in c for s in (2, 3))), 2):
    if sum(a != b for a, b in zip(x, y)) == 1:
        print(f"Answer: {''.join(a for a, b in zip(x, y) if a == b)}")
        break

Answer: xretqmmonskvzupalfiwhcfdb
Wall time: 16 ms


## Day 3
### Part One
How many square inches of fabric are within two or more claims?

In [50]:
%%time
expr = re.compile(r"#(\d+)\s@\s(\d+),(\d+):\s(\d+)x(\d+)")
fabric = np.zeros((1000, 1000), dtype=np.uint32)
with open(input_for_day(3), "r") as input_file:
    for claim in input_file:
        _, x, y, w, h = map(int, expr.match(claim).groups())
        fabric[y:y+h, x:x+w] += 1
    print(f"Answer: {np.sum(fabric > 1)}")

Answer: 117948
Wall time: 13 ms


### Part Two
What is the ID of the only claim that doesn't overlap?

In [55]:
%%time
expr = re.compile(r"#(\d+)\s@\s(\d+),(\d+):\s(\d+)x(\d+)")
fabric = np.zeros((1000, 1000), dtype=np.uint32)
with open(input_for_day(3), "r") as input_file:
    claims = [list(map(int, expr.match(claim).groups())) for claim in input_file]
for claim in claims:
    _, x, y, w, h = claim
    fabric[y:y+h, x:x+w] += 1
for claim in claims:
    claim_id, x, y, w, h = claim
    if np.all(fabric[y:y+h, x:x+w] == 1):
        print(f"Answer: {claim_id}")
        break

Answer: 567
Wall time: 15 ms


## Day 4
### Part One
What is the ID of the guard you chose multiplied by the minute you chose?